📝 **Author:** Amirhossein Heydari - 📧 **Email:** amirhosseinheydari78@gmail.com - 📍 **Linktree:** [linktr.ee/mr_pylin](https://linktr.ee/mr_pylin)

---

# Dependencies

In [1]:
import numpy as np
import pandas as pd
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, Dataset, TensorDataset
from torchvision import transforms

# Custom Classes in PyTorch
   - **PyTorch** is a **flexible** deep learning framework that allows developers to **customize** different components according to their **specific needs**.
   - This flexibility is essential for **implementing** custom **datasets**, **models**, and **optimization** routines, which may not be covered by the **built-in classes**.
---

## Load Breast Cancer Wisconsin (Diagnostic) Dataset

In [2]:
breast_cancer_dataset_url = r"https://github.com/mr-pylin/datasets/raw/refs/heads/main/data/tabular-data/breast-cancer-wisconsin-diagnostic/dataset.csv"

# pandas data-frame
df = pd.read_csv(breast_cancer_dataset_url, encoding='utf-8')

# log
df.head()

,ID,Diagnosis,radius1,texture1,perimeter1,area1,smoothness1,compactness1,concavity1,concave_points1,...,radius3,texture3,perimeter3,area3,smoothness3,compactness3,concavity3,concave_points3,symmetry3,fractal_dimension3
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [5]:
classes = df['Diagnosis'].unique()
class_to_idx = {l: i for i, l in enumerate(classes)}

# split dataset into features and labels
X, y = df.iloc[:, 2:].values, df.iloc[:, 1].values

# convert categorical labels into indices
y = np.array([class_to_idx[l] for l in y])

# properties of the dataset
num_samples, num_features = X.shape
classes, samples_per_class = np.unique(y, return_counts=True)

# log
print(f"X.shape: {X.shape}")
print(f"X.dtype: {X.dtype}")
print(f"y.shape: {y.shape}")
print(f"y.dtype: {y.dtype}")
print('-' * 50)
print(f"classes          : {classes}")
print(f"samples per class: {samples_per_class}")

X.shape: (569, 30)
X.dtype: float64
y.shape: (569,)
y.dtype: int32
--------------------------------------------------
classes          : [0 1]
samples per class: [212 357]


## Custom Dataset
   - PyTorch’s `Dataset` class can be easily subclassed to define custom datasets
   - This allows you to load and preprocess your data according to your needs.
   - Use `torch.utils.data.Dataset` as the parent class and override `__len__` and `__getitem__`.

📝 **Docs & Tutorials** 📚:
   - Data Loading Utility: [pytorch.org/docs/stable/data.html](https://pytorch.org/docs/stable/data.html)
   - Datasets & DataLoaders: [pytorch.org/tutorials/beginner/basics/data_tutorial.html](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html)

In [6]:
# convert numpy.ndarray to torch.Tensor
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32).view(-1, 1)

# log
print(f"type(X): {type(X)}  |  X.dtype: {X.dtype}  |  X.shape: {X.shape}")
print(f"type(y): {type(y)}  |  y.dtype: {y.dtype}  |  y.shape: {y.shape}")

type(X): <class 'torch.Tensor'>  |  X.dtype: torch.float32  |  X.shape: torch.Size([569, 30])
type(y): <class 'torch.Tensor'>  |  y.dtype: torch.float32  |  y.shape: torch.Size([569, 1])


In [27]:
class CustomDataset(Dataset):
    def __init__(self, data: torch.Tensor, labels: torch.Tensor) -> None:
        self.data = data
        self.labels = labels

    def __len__(self) -> int:
        return len(self.data)

    def __getitem__(self, index: int) -> tuple[torch.Tensor, torch.Tensor]:
        return self.data[index], self.labels[index]

In [28]:
# create a pytorch dataset
dataset_1 = CustomDataset(X, y)  # custom
dataset_2 = TensorDataset(X, y)  # built-in

# log
print(f"type(dataset_1) : {type(dataset_1)}")
print(f"len(dataset_1)  : {len(dataset_1)}")
print(f"dataset_1[0]    : {dataset_1[0]}")
print('-' * 50)
print(f"type(dataset_2) : {type(dataset_2)}")
print(f"len(dataset_2)  : {len(dataset_2)}")
print(f"dataset_2[0]    : {dataset_2[0]}")

type(dataset_1) : <class '__main__.CustomDataset'>
len(dataset_1)  : 569
dataset_1[0]    : (tensor([1.7990e+01, 1.0380e+01, 1.2280e+02, 1.0010e+03, 1.1840e-01, 2.7760e-01,
        3.0010e-01, 1.4710e-01, 2.4190e-01, 7.8710e-02, 1.0950e+00, 9.0530e-01,
        8.5890e+00, 1.5340e+02, 6.3990e-03, 4.9040e-02, 5.3730e-02, 1.5870e-02,
        3.0030e-02, 6.1930e-03, 2.5380e+01, 1.7330e+01, 1.8460e+02, 2.0190e+03,
        1.6220e-01, 6.6560e-01, 7.1190e-01, 2.6540e-01, 4.6010e-01, 1.1890e-01]), tensor([0.]))
--------------------------------------------------
type(dataset_2) : <class 'torch.utils.data.dataset.TensorDataset'>
len(dataset_2)  : 569
dataset_2[0]    : (tensor([1.7990e+01, 1.0380e+01, 1.2280e+02, 1.0010e+03, 1.1840e-01, 2.7760e-01,
        3.0010e-01, 1.4710e-01, 2.4190e-01, 7.8710e-02, 1.0950e+00, 9.0530e-01,
        8.5890e+00, 1.5340e+02, 6.3990e-03, 4.9040e-02, 5.3730e-02, 1.5870e-02,
        3.0030e-02, 6.1930e-03, 2.5380e+01, 1.7330e+01, 1.8460e+02, 2.0190e+03,
        1.622

## Custom Transform
   - Transforms are used to modify the input data before feeding it into the model.
   - PyTorch provides a lot of built-in transforms (like cropping, flipping, etc.) in `torchvision.transforms`.
   - you can define your own transformation by implementing the `__call__` method.

📝 **Docs & Tutorials** 📚:
   - Transforming and augmenting images: [pytorch.org/vision/stable/transforms.html](https://pytorch.org/vision/stable/transforms.html)
   - Transforms: [pytorch.org/tutorials/beginner/basics/transforms_tutorial.html](https://pytorch.org/tutorials/beginner/basics/transforms_tutorial.html)

In [29]:
class NumpyToTensor():
    def __call__(self, sample: np.ndarray) -> tuple[torch.Tensor, torch.Tensor]:
        converted_sample = torch.tensor(sample[0], dtype=torch.float32), torch.tensor(sample[1], dtype=torch.float32)
        return converted_sample

In [30]:
class NormalizeTo01():
    def __init__(self) -> None:
        self.min_val = None
        self.max_val = None

    def fit(self, data: np.ndarray) -> None:
        self.min_val = np.min(data, axis=0).astype(np.float32)
        self.max_val = np.max(data, axis=0).astype(np.float32)

    def __call__(self, sample: tuple[torch.Tensor, torch.Tensor]):
        normalized_sample = (sample[0] - self.min_val) / (self.max_val - self.min_val), sample[1]
        return normalized_sample

### Direct transform

In [31]:
# dataset
X = np.array([[1, 2, 3], [5, 1, 2], [3, 3, 3]])
y = np.array([[0], [0], [1]])
dataset = list(zip(X, y))

# log
print(f"dataset : {dataset}")

dataset : [(array([1, 2, 3]), array([0])), (array([5, 1, 2]), array([0])), (array([3, 3, 3]), array([1]))]


In [32]:
# NumpyToTensor
t_totensor = NumpyToTensor()

# NormalizeTo01
t_normalize = NormalizeTo01()
t_normalize.fit(X)

# transform the first input
result = t_normalize(t_totensor(dataset[0]))

# log
print(f"result          : {result}")
print(f"result[0].dtype : {result[0].dtype}")
print(f"result[1].dtype : {result[1].dtype}")

result          : (tensor([0.0000, 0.5000, 1.0000]), tensor([0.]))
result[0].dtype : torch.float32
result[1].dtype : torch.float32


### Integrated transform

In [33]:
# advanced Dataset with transform support
class AdvancedCustomDataset(Dataset):
    def __init__(self, data, labels, transform=None):
        self.data = data
        self.labels = labels
        self.transform = transform

        # if NormalizeTo01 is included, call <fit> method for that
        if self.transform:
            for t in self.transform.transforms:
                if isinstance(t, NormalizeTo01):
                    t.fit(self.data)
                    break

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        sample = self.data[index], self.labels[index]

        if self.transform:
            sample = self.transform(sample)

        return sample

In [34]:
X = np.array([[1, 2, 3], [5, 1, 2], [3, 3, 3]])
y = np.array([[0], [0], [1]])

transformations = transforms.Compose([
    NumpyToTensor(),
    NormalizeTo01(),
])

dataset = AdvancedCustomDataset(X, y, transformations)

# log
for i in range(len(y)):
    print(f"dataset[{i}]: {dataset[i]}")
    print(f"    -> input data : {dataset[i][0]}")
    print(f"    -> label      : {dataset[i][1]}", end='\n\n')

dataset[0]: (tensor([0.0000, 0.5000, 1.0000]), tensor([0.]))
    -> input data : tensor([0.0000, 0.5000, 1.0000])
    -> label      : tensor([0.])

dataset[1]: (tensor([1., 0., 0.]), tensor([0.]))
    -> input data : tensor([1., 0., 0.])
    -> label      : tensor([0.])

dataset[2]: (tensor([0.5000, 1.0000, 1.0000]), tensor([1.]))
    -> input data : tensor([0.5000, 1.0000, 1.0000])
    -> label      : tensor([1.])



## Custom Activation Function
   - you can create your own activation function by subclassing `torch.nn.Module`.
   - Use `torch.nn.Module` as the parent class and implement  `forward` method.

📝 **Docs & Tutorials** 📚:
   - Non-linear Activations (weighted sum, nonlinearity): [pytorch.org/docs/stable/nn.html#non-linear-activations-weighted-sum-nonlinearity](https://pytorch.org/docs/stable/nn.html#non-linear-activations-weighted-sum-nonlinearity)
   - Non-linear Activations (other): [pytorch.org/docs/stable/nn.html#non-linear-activations-other](https://pytorch.org/docs/stable/nn.html#non-linear-activations-other)
   - Non-linear activation functions: [pytorch.org/docs/stable/nn.functional.html#non-linear-activation-functions](https://pytorch.org/docs/stable/nn.functional.html#non-linear-activation-functions)

In [35]:
class CustomSigmoid(nn.Module):
    def __init__(self):
        super(CustomSigmoid, self).__init__()

    def forward(self, x):
        return torch.sigmoid(x)

In [36]:
sig_1 = CustomSigmoid()
sig_2 = nn.Sigmoid()

values = torch.tensor([10, 0, -10], dtype=torch.float32)

# log
print(f"sig_1(values) : {sig_1(values)}")
print(f"sig_2(values) : {sig_2(values)}")

sig_1(values) : tensor([9.9995e-01, 5.0000e-01, 4.5398e-05])
sig_2(values) : tensor([9.9995e-01, 5.0000e-01, 4.5398e-05])


## Custom Model
   - Sequential Model:
      - Useful for simpler models where the layers are stacked in a linear sequence
      - The `torch.nn.Sequential` class allows you to stack layers in a sequence, passing the output of one layer directly to the next.
      - This is great for simple models like fully-connected neural networks or basic CNNs.
      - Key Points
         - Layers are defined in the order they are passed to `Sequential`.
         - You don't need to define the `forward` method manually; PyTorch handles it for you.
   - Functional Model:
      - Allowing for complex architectures where you might need non-linear layer connections (e.g., skip connections in ResNet)
      - models are created by subclassing `torch.nn.Module`.
      - This allows you to define any neural network architecture, from simple feedforward networks to complex architectures like GANs or transformers
      - Key Points
         - Use `torch.nn.Module` as the parent class and implement  `forward` method.

📝 **Docs & Tutorials** 📚:
   - Module: [pytorch.org/docs/stable/generated/torch.nn.Module.html](https://pytorch.org/docs/stable/generated/torch.nn.Module.html)
   - torch.nn: [pytorch.org/docs/stable/nn.html](https://pytorch.org/docs/stable/nn.html)
   - Building Models with PyTorch: [pytorch.org/tutorials/beginner/introyt/modelsyt_tutorial.html](https://pytorch.org/tutorials/beginner/introyt/modelsyt_tutorial.html)
   - Build the Neural Network: [pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html](https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html)
   - Neural Networks: [pytorch.org/tutorials/beginner/blitz/neural_networks_tutorial](https://pytorch.org/tutorials/beginner/blitz/neural_networks_tutorial)

In [37]:
# a simple sequential model
model_1 = nn.Sequential(
    nn.Linear(in_features=30, out_features=16),
    nn.Sigmoid(),
    nn.Linear(in_features=16, out_features=1),
    nn.Sigmoid()
)

# log
model_1

Sequential(
  (0): Linear(in_features=30, out_features=16, bias=True)
  (1): Sigmoid()
  (2): Linear(in_features=16, out_features=1, bias=True)
  (3): Sigmoid()
)

In [38]:
# a simple functional model
class CustomLogisticRegression(nn.Module):
    def __init__(self, input_size: int, hidden_size: int, output_size: int) -> None:
        super(CustomLogisticRegression, self).__init__()

        self.fc1 = nn.Linear(in_features=input_size, out_features=hidden_size)
        self.sigmoid1 = CustomSigmoid()
        self.fc2 = nn.Linear(in_features=hidden_size, out_features=output_size)
        self.sigmoid2 = CustomSigmoid()

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.fc1(x)
        x = self.sigmoid1(x)
        x = self.fc2(x)
        x = self.sigmoid2(x)
        return x

# initialize the model
model_2 = CustomLogisticRegression(30, 16, 1)

# log
model_2

CustomLogisticRegression(
  (fc1): Linear(in_features=30, out_features=16, bias=True)
  (sigmoid1): CustomSigmoid()
  (fc2): Linear(in_features=16, out_features=1, bias=True)
  (sigmoid2): CustomSigmoid()
)

## Custom Loss Function
   - PyTorch comes with standard loss functions like MSE, Cross-Entropy, etc.
   - you can create your own loss function by subclassing `torch.nn.Module`.
   - Use `torch.nn.Module` as the parent class and implement  `forward` method.

📝 **Docs & Tutorials** 📚:
   - Loss Functions: [pytorch.org/docs/stable/nn.html#loss-functions](https://pytorch.org/docs/stable/nn.html#loss-functions)

In [39]:
class CustomMSE(nn.Module):
    def __init__(self):
        super(CustomMSE, self).__init__()

    def forward(self, y_pred, y_true):
        loss = torch.mean((y_pred - y_true) ** 2)
        return loss

In [40]:
# split dataset into (data, labels)
X = dataset_1[:][0]
y_true = dataset_1[:][1]

# feed-forward
y_pred = model_1(X)

# MSE loss function
criterion_1 = CustomMSE()   # custom
criterion_2 = nn.MSELoss()  # built-in

# compute the loss
loss_1 = criterion_1(y_pred, y_true)
loss_2 = criterion_2(y_pred, y_true)

# log
print(f"loss_1: {loss_1}")
print(f"loss_2: {loss_2}")

loss_1: 0.25902360677719116
loss_2: 0.25902360677719116


## Custom Optimizer
   - PyTorch offers optimizers like SGD, Adam, etc.
   - you can create your own optimizer by subclassing `torch.optim.Optimizer`.
   - Use `torch.optim.Optimizer` as the parent class and override `step` method.

📝 **Docs & Tutorials** 📚:
   - torch.optim: [pytorch.org/docs/stable/optim.html](https://pytorch.org/docs/stable/optim.html)
   - torch.optim.Optimizer.step: [pytorch.org/docs/stable/generated/torch.optim.Optimizer.step.html](https://pytorch.org/docs/stable/generated/torch.optim.Optimizer.step.html)

In [41]:
# this implementation might not be the same as SGD
class CustomSGD(optim.Optimizer):
    def __init__(self, params, lr=0.01, momentum=0):
        defaults = dict(lr=lr, momentum=momentum)
        super(CustomSGD, self).__init__(params, defaults)

    def step(self):
        for group in self.param_groups:
            lr = group['lr']
            momentum = group['momentum']

            for p in group['params']:
                if p.grad is None:
                    continue
                d_p = p.grad.data

                if momentum != 0:
                    param_state = self.state[p]
                    if 'momentum_buffer' not in param_state:
                        buf = param_state['momentum_buffer'] = torch.zeros_like(p.data)
                        buf.mul_(momentum).add_(d_p)
                    else:
                        buf = param_state['momentum_buffer']
                        buf.mul_(momentum).add_(d_p, alpha=1 - momentum)
                    d_p = buf

                p.data.add_(d_p, alpha=-lr)

In [42]:
optimizer_1 = CustomSGD(model_1.parameters())
optimizer_2 = optim.SGD(model_1.parameters())

# log
print(f"optimizer_1:\n{optimizer_1}\n")
print(f"optimizer_2:\n{optimizer_2}")

optimizer_1:
CustomSGD (
Parameter Group 0
    lr: 0.01
    momentum: 0
)

optimizer_2:
SGD (
Parameter Group 0
    dampening: 0
    differentiable: False
    foreach: None
    fused: None
    lr: 0.001
    maximize: False
    momentum: 0
    nesterov: False
    weight_decay: 0
)


# Example: All In One

In [43]:
# load breast-cancer dataset
X, y = load_breast_cancer(return_X_y=True)

# create a custom Dataset class
class CustomDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        return self.data[index], self.labels[index]


# convert numpy.ndarray to torch.Tensor
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32).view(-1, 1)

# create a dataset
dataset = CustomDataset(X, y)

# create a dataloader
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

In [44]:
# custom sigmoid activation
class CustomSigmoid(nn.Module):
    def __init__(self):
        super(CustomSigmoid, self).__init__()

    def forward(self, x):
        return torch.sigmoid(x)

# model
class CustomLogisticRegression(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(CustomLogisticRegression, self).__init__()

        self.fc1 = nn.Linear(in_features=input_size, out_features=hidden_size)
        self.sigmoid1 = CustomSigmoid()
        self.fc2 = nn.Linear(in_features=hidden_size, out_features=output_size)
        self.sigmoid2 = CustomSigmoid()

    def forward(self, x):
        x = self.fc1(x)
        x = self.sigmoid1(x)
        x = self.fc2(x)
        x = self.sigmoid2(x)
        return x


input_size = X.shape[1]
hidden_size = 2
output_size = y.shape[1]

# model
model = CustomLogisticRegression(input_size, hidden_size, output_size)
model

CustomLogisticRegression(
  (fc1): Linear(in_features=30, out_features=2, bias=True)
  (sigmoid1): CustomSigmoid()
  (fc2): Linear(in_features=2, out_features=1, bias=True)
  (sigmoid2): CustomSigmoid()
)

In [45]:
# hyper parameters
epochs = 10
lr = 0.005
criterion = CustomMSE()
optimizer = CustomSGD(model.parameters(), lr=lr)

In [46]:
# training loop
model.train()
total_loss = []
total_acc = []

for epoch in range(epochs):

    epoch_loss = 0
    epoch_acc = 0

    for x, y_true in dataloader:

        # forward
        y_pred = model(x)
        loss = criterion(y_pred, y_true)

        # backward
        loss.backward()

        # update parameters
        optimizer.step()
        optimizer.zero_grad()

        # store loss & accuracy
        epoch_loss += loss.item()
        epoch_acc += ((y_pred > .5).float() == y_true).sum().item()

    total_loss.append(epoch_loss / len(dataloader))
    total_acc.append(epoch_acc / len(X))

    # log
    print(f"epoch {epoch:>2}  ->  loss: {total_loss[-1]:.5f} - accuracy: {total_acc[-1] * 100:.2f}%")

epoch  0  ->  loss: 0.34326 - accuracy: 37.26%
epoch  1  ->  loss: 0.34738 - accuracy: 37.26%
epoch  2  ->  loss: 0.25628 - accuracy: 37.26%
epoch  3  ->  loss: 0.25216 - accuracy: 38.14%
epoch  4  ->  loss: 0.24787 - accuracy: 62.74%
epoch  5  ->  loss: 0.24429 - accuracy: 62.74%
epoch  6  ->  loss: 0.24261 - accuracy: 62.74%
epoch  7  ->  loss: 0.24102 - accuracy: 62.74%
epoch  8  ->  loss: 0.23829 - accuracy: 62.74%
epoch  9  ->  loss: 0.23698 - accuracy: 62.74%
